# Example of a Full Data Cleaning and Model Fitting Pipeline

In [1]:
# Import libraries
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_style('darkgrid')